#Introduction
This notebook performs an initial analysis of airline flight price data. The steps taken include:

1.  **Cloning a GitHub repository**: The notebook starts by cloning a GitHub repository containing the raw flight data.
2.  **Loading and combining data**: It then loads multiple CSV files from the cloned repository's data directory into a single pandas DataFrame, ensuring that date columns are correctly parsed.
3.  **Creating a Unique Identifier**: A `UniqueID` column is generated by combining relevant columns to uniquely identify each flight offer.
4.  **Grouping Flight Offers**: The data is grouped by the `UniqueID` to aggregate information for each unique flight offer, handling cases where offers span multiple rows.
5.  **Data Validation**: Basic data validation is performed to check for missing values, empty strings in object columns, zeros in numeric columns, and duplicate rows.
6. **Save CSV**: Saves data as a CSV and pushes the cleaned dataframe to github repo.

In [ ]:
# Go to the content folder
%cd /content

# Remove old repo if it exists
!rm -rf Airline-Flight-Price-Analysis-with-APIs-Azure

# Clone the latest version of your repo
!git clone https://github.com/williamervin7/Airline-Flight-Price-Analysis-with-APIs-Azure.git

# Check contents
!ls Airline-Flight-Price-Analysis-with-APIs-Azure

/content
Cloning into 'Airline-Flight-Price-Analysis-with-APIs-Azure'...
remote: Enumerating objects: 289, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (82/82), done.
remote: Total 289 (delta 52), reused 18 (delta 10), pack-reused 177 (from 1)
Receiving objects: 100% (289/289), 135.30 KiB | 5.01 MiB/s, done.
Resolving deltas: 100% (129/129), done.
data  figures  notebooks  README.md  scripts


### Read in raw data
The code in the cell below loads data from multiple CSV files within the `Airline-Flight-Price-Analysis-with-APIs-Azure/data/raw` directory into a single pandas DataFrame.

Here's a breakdown:

1.  **Import libraries**: Imports `os` for interacting with the operating system (like listing files) and `pandas` for data manipulation.
2.  **Define data path**: Sets the variable `data_path` to the location of the raw data files.
3.  **List files**: Gets a sorted list of all files in the specified `data_path`.
4.  **Initialize empty list**: Creates an empty list `dfs` to store individual DataFrames.
5.  **Iterate and load CSVs**: Loops through each file in the `files` list.
    *   It checks if the file ends with `.csv`.
    *   It constructs the full path to the file.
    *   It prints a message indicating which file is being loaded.
    *   It reads the CSV file into a pandas DataFrame using `pd.read_csv`, parsing the specified date columns.
    *   It appends the loaded DataFrame to the `dfs` list.
6.  **Concatenate DataFrames**: Combines all DataFrames in the `dfs` list into a single DataFrame called `all_data`, ignoring the original index.
7.  **Print summary**: Prints a message indicating the number of files loaded and the total number of rows in the combined DataFrame.

In [ ]:
import os
import pandas as pd

# Path to my data folder
data_path = 'Airline-Flight-Price-Analysis-with-APIs-Azure/data/raw'

# Get list of files in folder
files = sorted(os.listdir(data_path))

# Collect all dataframes
dfs = []

for f in files:
  if f.endswith('.csv'): #only load CSV
    full_path = os.path.join(data_path, f)
    print(f"Loading {full_path}...")

    df = pd.read_csv(full_path, parse_dates=['Departure', 'Arrival', 'DepartureDate', 'SearchDate'])
    dfs.append(df)

all_data = pd.concat(dfs, ignore_index=True)
print(f'Loaded {len(files)-1} files, {all_data.shape[0]} rows')

Loading Airline-Flight-Price-Analysis-with-APIs-Azure/data/raw/2025-10-03.csv...
Loading Airline-Flight-Price-Analysis-with-APIs-Azure/data/raw/2025-10-04.csv...
Loading Airline-Flight-Price-Analysis-with-APIs-Azure/data/raw/2025-10-05.csv...
Loaded 3 files, 955 rows


In [ ]:
print(all_data.head())

   OfferID Airline  Flight From           Departure   To             Arrival  \
0        1      UA    1743  IAH 2025-11-15 16:30:00  LAX 2025-11-15 18:10:00   
1        2      UA    2460  IAH 2025-11-15 12:00:00  LAX 2025-11-15 13:45:00   
2        3      UA    1945  IAH 2025-11-15 14:30:00  LAX 2025-11-15 16:15:00   
3        4      UA     434  IAH 2025-11-15 21:35:00  LAX 2025-11-15 23:21:00   
4        5      UA    2431  IAH 2025-11-15 05:42:00  LAX 2025-11-15 07:30:00   

  Duration   Price DepartureDate SearchDate  
0  PT3H40M  107.64    2025-11-15 2025-10-03  
1  PT3H45M  107.64    2025-11-15 2025-10-03  
2  PT3H45M  107.64    2025-11-15 2025-10-03  
3  PT3H46M  107.64    2025-11-15 2025-10-03  
4  PT3H48M  107.64    2025-11-15 2025-10-03  


### Create a `UniqueID`
The code in the cell below creates a new column called `UniqueID` in the `all_data` DataFrame. This unique ID is generated by concatenating the `OfferID`, `DepartureDate`, and `SearchDate` columns, separated by underscores. This creates a unique identifier for each flight offer on a specific departure date that was searched on a particular date.

In [ ]:
# UniqueID = SearchDate + DepartureDate + origin + destination + OfferID
all_data['UniqueID'] = (all_data['OfferID'].astype(str) +'_' + all_data['DepartureDate'].astype(str) +'_' + all_data['SearchDate'].astype(str))

###Group Offers
Some offers are on multiple rows so we need to group them. The code in the cell below groups the `all_data` DataFrame by the `UniqueID` column and aggregates the data to create a summary DataFrame called `trip_df`.

Here's a breakdown of the aggregation:

*   **`airline`**: Takes the first value of the `Airline` column for each `UniqueID`.
*   **`origin`**: Takes the first value of the `From` column for each `UniqueID`.
*   **`destination`**: Takes the last value of the `To` column for each `UniqueID`.
*   **`departure_time`**: Takes the first value of the `Departure` column for each `UniqueID`.
*   **`Arrival_time`**: Takes the last value of the `Arrival` column for each `UniqueID`.
*   **`price`**: Takes the first value of the `Price` column for each `UniqueID`.
*   **`flights`**: Creates a list of all `Flight` values for each `UniqueID`.
*   **`DepartureDate`**: Takes the first value of the `DepartureDate` column for each `UniqueID`.
*   **`SearchDate`**: Takes the first value of the `SearchDate` column for each `UniqueID`.

Finally, `.reset_index()` converts the `UniqueID` from the index back into a column.

In [ ]:
trip_df = (
    all_data.groupby(['UniqueID']).agg(
        airline=('Airline','first'),
        origin=('From','first'),
        destination=('To','last'),
        departure_time=('Departure','first'),
        Arrival_time=('Arrival','last'),
        price=('Price','first'),
        flights=('Flight', lambda x: list(map(str, x))),
        DepartureDate=('DepartureDate','first'),
        SearchDate=('SearchDate','first'),
    ).reset_index()
    )

In [ ]:
print(trip_df.head())

                  UniqueID airline origin destination      departure_time  \
0  1_2025-11-15_2025-10-03      UA    IAH         LAX 2025-11-15 16:30:00   
1  1_2025-11-15_2025-10-04      UA    IAH         LAX 2025-11-15 16:30:00   
2  1_2025-11-15_2025-10-05      UA    IAH         LAX 2025-11-15 16:30:00   
3  1_2025-11-16_2025-10-03      UA    IAH         LAX 2025-11-16 22:00:00   
4  1_2025-11-16_2025-10-04      UA    IAH         LAX 2025-11-16 22:00:00   

         Arrival_time   price flights DepartureDate SearchDate  
0 2025-11-15 18:10:00  107.64  [1743]    2025-11-15 2025-10-03  
1 2025-11-15 18:10:00   77.56  [1743]    2025-11-15 2025-10-04  
2 2025-11-15 18:10:00   77.56  [1743]    2025-11-15 2025-10-05  
3 2025-11-16 23:24:00  107.64  [1851]    2025-11-16 2025-10-03  
4 2025-11-16 23:24:00   77.56  [1851]    2025-11-16 2025-10-04  


In [ ]:
print(trip_df.isna().sum())

UniqueID          0
airline           0
origin            0
destination       0
departure_time    0
Arrival_time      0
price             0
flights           0
DepartureDate     0
SearchDate        0
dtype: int64


In [ ]:
print(trip_df.dtypes)

UniqueID                  object
airline                   object
origin                    object
destination               object
departure_time    datetime64[ns]
Arrival_time      datetime64[ns]
price                    float64
flights                   object
DepartureDate     datetime64[ns]
SearchDate        datetime64[ns]
dtype: object


In [ ]:
# Check for empty strings in object-type columns
for col in trip_df.select_dtypes(include=['object']).columns:
  empty_count = (trip_df[col].str.strip() == '').sum()
  print(f"Empty strings in '{col}': {empty_count}")

Empty strings in 'UniqueID': 0
Empty strings in 'airline': 0
Empty strings in 'origin': 0
Empty strings in 'destination': 0
Empty strings in 'flights': 0


In [ ]:
# Check for zeros in numeric columns
for col in trip_df.select_dtypes(include='number').columns:
  zero_count = (trip_df[col] == 0).sum()
  print(f"Zeros in '{col}': {zero_count}")

Zeros in 'price': 0


In [ ]:
duplicate_count = trip_df.drop(columns=['flights']).duplicated().sum()
print(f"Duplicate rows: {duplicate_count}")

Duplicate rows: 0


In [ ]:
trip_df.to_csv('cleaned_fight_offers.csv', index=False)